# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from sql_queries import insert_data_queries

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

all_filepath=[]

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
#     print(root, dirs, files)
#     print('\n')
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    all_filepath.append(file_path_list)
#     print(all_filepath)
all_filepath = all_filepath[0][:]
all_filepath.sort()
# all_filepath

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
song_all = []
for n, f in enumerate(all_filepath):
    
    df = pd.read_csv(f)
    
    song = df[['artist','firstName','gender','itemInSession','lastName','length',
               'level','location','sessionId','song','userId']]
    song = song.dropna(how='any')
    
    if n > 0:
        song_all = pd.concat([song_all, song], axis=0)
    else:
        song_all = song
        
song_all.to_csv("event_datafile_new.csv", encoding="utf-8", index=False)

#### Check how many rows will be in the csv file created

In [ ]:
len(song_all)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_songs 
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}""")

#### Set Keyspace

In [ ]:
session.set_keyspace('music_songs')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1: 
#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Create table

In [ ]:
query = "CREATE TABLE IF NOT EXISTS song_per_session"
query = query +  """(sessionId int, iteminsession int, artist text, song text, 
length float, PRIMARY KEY((sessionId, iteminsession)))"""
session.execute(query)

#### Insert data

In [ ]:
file = 'event_datafile_new.csv'
#read the csv by pandas
dfnew = pd.read_csv(file, encoding = 'utf8')
df1 = dfnew[['sessionId','itemInSession','artist','song','length']]

# insert records
for i, row in df1.iterrows():
    session.execute(insert_data_queries[0], (row.sessionId, row.itemInSession, row.artist, row.song, row.length))
    if i%1000==0:
        print("*")
print("Inserted to 1st table")

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query_select = "SELECT sessionId, iteminsession, artist, song, length FROM song_per_session WHERE sessionId = 338 AND iteminsession = 4"

try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
for r in rows:
    print("sessionId:{} | iteminsession:{} | artist:{} | song:{} | length: {}".format(r.sessionid, r.iteminsession,r.artist, r.song, r.length))
    print("*"*70)

### Query 2: 
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create table

In [ ]:
query = "CREATE TABLE IF NOT EXISTS user_listened"
query = query +  "(userId int, sessionId int, iteminsession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), iteminsession))"
session.execute(query)

#### Insert data

In [ ]:
file = 'event_datafile_new.csv'
df2 = dfnew[['userId' , 'sessionId', 'itemInSession', 'artist', 'song', 'firstName' , 'lastName']]
# insert records
for i, row in df2.iterrows():
    session.execute(insert_data_queries[1], (int(row.userId) , row.sessionId, row.itemInSession, row.artist, row.song, row.firstName , row.lastName))
    if i%1000==0:
        print("*")
print("Inserted to 2nd table")

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query_select2 = "SELECT artist, song, firstname, lastname FROM user_listened WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query_select2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname , row.lastname )

### Query 3: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create table

In [ ]:
query = "CREATE TABLE IF NOT EXISTS wholistened_theSong"
query = query +  "(song text, userId int, firstName text, lastName text, PRIMARY KEY((song, userId)))"
session.execute(query)

#### Insert data

In [ ]:
file = 'event_datafile_new.csv'

dfnew = pd.read_csv(file, encoding = 'utf8')

df3 = dfnew[['song', 'userId', 'firstName', 'lastName']]
df3.head(20)

for i, row in df3.iterrows():
    
    session.execute(insert_data_queries[2], (row.song, int(row.userId), row.firstName, row.lastName))

    if i%1000==0:
        print("*")

print("Inserted into 3rd table")

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query_select3 = "SELECT  firstname , lastname \
FROM wholistened_theSong WHERE song = 'All Hands Against His Own' and userId = 80"
try:
    rows = session.execute(query_select3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname,  row.lastname)

### Drop the tables before closing out the sessions

In [ ]:
session.execute("drop table song_per_sessiong")
session.execute("drop table user_listened")
session.execute("drop table wholistened_theSong")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()